In [ ]:
import numpy as np
import cv2

from scipy.ndimage.filters import convolve

In [ ]:
# prevents overflow for float 64
def overflow_64(img):
    img[img>=65535] = 65535
    img[img<=0] = 0

    return img

In [ ]:
# prevents overflow for uint8
def overflow_8(img):
    img[img>=255] = 255
    img[img<=0] = 0

    return img

In [ ]:
# create cfa_masks
def cfa_mask(img, pattern):
    height, width = img.shape[0], img.shape[1]

    mask_R = np.zeros(img.shape)
    mask_G = np.zeros(img.shape)
    mask_B = np.zeros(img.shape)

    if pattern == 'RGGB':
        for i in range(height):
            for j in range(width):
                if i%2==0 and j%2==0:
                    mask_R[i][j] = 1
                if i%2==0 and j%2==1:
                    mask_G[i][j] = 1
                if i%2==1 and j%2==0:
                    mask_G[i][j] = 1
                if i%2==1 and j%2==1:
                    mask_B[i][j] = 1

    elif pattern == 'GRBG':
        for i in range(height):
            for j in range(width):
                if i%2==0 and j%2==1:
                    mask_R[i][j] = 1
                if i%2==0 and j%2==0:
                    mask_G[i][j] = 1
                if i%2==1 and j%2==1:
                    mask_G[i][j] = 1
                if i%2==1 and j%2==0:
                    mask_B[i][j] = 1

    return mask_R, mask_G, mask_B

In [ ]:
# bilinear demosaicing algorithm
# RGGB for horizontal, GRBG for vertical img
# BGR stack for horizontal, RGB stack for vertical img
def cfa_interpolation(img):    
    height, width = img.shape[0], img.shape[1]

    kernel_RB = (np.array([
        [1,2,1],
        [2,4,2], 
        [1,2,1]
    ])/4)
    
    kernel_G = (np.array([
        [0,1,0],
        [1,4,1],
        [0,1,0]
    ])/4) 
    
    if height > width :
        mask_R, mask_G, mask_B = cfa_mask(img, 'RGGB')

        R = convolve(img*mask_R, kernel_RB)
        G = convolve(img*mask_G, kernel_G)
        B = convolve(img*mask_B, kernel_RB)

        result = np.stack([B,G,R], axis=-1)
    else :
        mask_R, mask_G, mask_B = cfa_mask(img, 'GRBG')

        R = convolve(img*mask_R, kernel_RB)
        G = convolve(img*mask_G, kernel_G)
        B = convolve(img*mask_B, kernel_RB)

        result = np.stack([R,G,B], axis=-1)
    
    return result

In [ ]:
# automatic white balance based on grey world algorithm
def automatic_WB(img):    
    avg_ch0 = img[:,:,0].mean()
    avg_ch1 = img[:,:,1].mean()
    avg_ch2 = img[:,:,2].mean()

    ch0 = avg_ch1/avg_ch0 * img[:,:,0]
    ch1 = img[:,:,1]
    ch2 = avg_ch1/avg_ch2 * img[:,:,2]

    result = np.stack([ch0,ch1,ch2], axis=-1)

    return result

In [ ]:
# gamma correction
# for scale [0, 255]
def gamma_correction(img, gamma=2.2):
    result = ((img/65536)**(1/gamma))*255
    
    return result

In [ ]:
def sharpen(img, value=1):
    kernel = np.array([[0, -1, 0],
                       [-1, 5,-1],
                       [0, -1, 0]])

    ch0 = img[:,:,0]
    ch1 = img[:,:,1]
    ch2 = img[:,:,2]
    
    for i in range(value):
        ch0 = convolve(ch0, kernel)
        ch1 = convolve(ch1, kernel)
        ch2 = convolve(ch2, kernel)

    result = np.stack([ch0,ch1,ch2], axis=-1)
    
    return result

In [ ]:
def contrast(img, value=1):
    ch0 = img[:,:,0] * value
    ch1 = img[:,:,1] * value
    ch2 = img[:,:,2] * value

    result = np.stack([ch0,ch1,ch2], axis=-1)
    
    return result

In [ ]:
def brightness(img, value=0):
    ch0 = img[:,:,0] + value
    ch1 = img[:,:,1] + value
    ch2 = img[:,:,2] + value

    result = np.stack([ch0,ch1,ch2], axis=-1)
    
    return result

In [ ]:
def compression(img):
    result = overflow_8(img)
    result = img.astype(np.uint8)
    
    return result

In [ ]:
# select img_type(e.g. 'picture', 'document')
def process_image(tiff, output_name, img_type):
    output = cfa_interpolation(tiff)
    output = automatic_WB(output)
    output = gamma_correction(output)

    if img_type == "document":
        output = sharpen(output, 1)
        output = contrast(output, 2.1)
        output = brightness(output, -110)

    elif img_type == "picture":        
        output = sharpen(output, 0)
        output = contrast(output, 1.5)
        output = brightness(output, -40)

    result = compression(output)
    cv2.imsave(output_name, result)

    return result

In [ ]:
# read TIFFs
# given images
sample_0 = cv2.imread('./test_images/20200917_170725.tiff')
sample_1 = cv2.imread('./test_images/20200917_170752.tiff')
# custom images
sample_2 = cv2.imread('./test_images/20221004_102514.tiff')
sample_3 = cv2.imread('./test_images/20221004_103335.tiff')
sample_4 = cv2.imread('./test_images/20221004_103548.tiff')
sample_5 = cv2.imread('./test_images/20221004_103814.tiff')

#pipeline: Gain Control A/D Converter Possible LUT -> White Balance -> CFA Demosaicing -> Noise Reduction/Sharpening -> Color Space Transform + Color Preferences -> Tone Reproduction -> JPEG Compression -> Exif File Info -> Save to storage 
# given images
jpeg_0 = process_image(sample_0, 'sample_0.jpeg', 'picture')
jpeg_1 = process_image(sample_1, 'sample_1.jpeg', 'document')
# custom images
jpeg_2 = process_image(sample_2, 'sample_2.jpeg', 'picture')
jpeg_3 = process_image(sample_3, 'sample_3.jpeg', 'picture')
jpeg_4 = process_image(sample_4, 'sample_4.jpeg', 'picture')
jpeg_5 = process_image(sample_5, 'sample_5.jpeg', 'document')